# Finding an object in an image using template matching

## Compiler parameters

Set my Jypyter environment for the use of [OpenCV](https://www.opencv.org/) in a [C++ notebook](https://github.com/jupyter-xeus/xeus-cling). You don't need this line when you write yur own C++ programs. I need it to set my interactive compiler ([Cling](https://root.cern/cling/)). For your own program, use [CMake](https://www.cmake.org/).

In [1]:
#include "../../../includeLibraries.h"

## Header inclusion for C++

In [2]:
#include <iostream>
#include <cstdlib>
#include <vector>
#include <tuple>
#include <stdexcept>
#include <sstream>
// #include <string>
#include <limits>
#include <opencv2/opencv.hpp>
#include <opencv2/tracking.hpp> // selectROI is part of tracking API

## Add the namespaces

In [3]:
using namespace std;

In [4]:
using namespace cv;

## Open the image

![Test image](../example1/cells.png)

In [5]:
Mat test_image = imread("../example1/cells.png", IMREAD_COLOR);

if (!test_image.data)
    throw runtime_error("Could not find the image, the program will terminate");

## Open the template

![Template](../example1/template.jpg)

In [6]:
Mat template_image = imread("../example1/template.jpg", IMREAD_COLOR);

if (!template_image.data)
    throw runtime_error("Could not find the image, the program will terminate");

Write a function to apply the zero-mean, unit-variance normalisation

$Img_2(x,y) = \frac{Img_1(x,y) - \mathrm{avg}(Img_1)}{\mathrm{stddev}(Img_1)}$

In [7]:
Mat zeroMeanUnitVarianceNormalisation(const Mat& anImage)
{
    Scalar average;
    Scalar stddev;

    Mat image;
    anImage.copyTo(image);
    cvtColor(image, image, COLOR_RGB2GRAY);
    image.convertTo(image, CV_32F);

    meanStdDev(image, average, stddev, cv::Mat());
        
    return (image - average) / stddev;
}

The average pixel value of `normalised_patch` is zero, the variance is 1.

In [8]:
Mat normalised_template = zeroMeanUnitVarianceNormalisation(template_image);

## Compute the cross-correlation function

Create a new image of the right size

1. For every pixel location,
2. Extract the corresponding region of interest (ROI)
3. Transform it using the zero-mean, unit-variance normalisation
4. Perform a pixel-wise product between the two normalised patches
5. Compute its mean pixel value: It corresponds to the zero-mean normalised cross-correlation (ZNCC)
6. This is the auto-correlation value for that pixel location
7. The auto-correlation values are between -1 and 1.

In [9]:
Mat autocorrelation_float(test_image.rows,
                    test_image.cols,
                    CV_32FC1, 
                    Scalar(0,0,0));

Mat autocorrelation_uchar(test_image.rows,
                    test_image.cols,
                    CV_8UC3, 
                    Scalar(128,128,128));


// 1. For every pixel location,
for (int j = 0; j < autocorrelation_float.rows - template_image.rows; ++j)
{
    for (int i = 0; i < autocorrelation_float.cols - template_image.cols; ++i)
    {
        // 2. Extract the corresponding region of interest (ROI)
        //Rect roi(i, j, template_image.cols, template_image.rows); The kernel is being funny
        Rect roi;
        roi.x = i;
        roi.y = j;
        roi.width = template_image.cols;
        roi.height = template_image.rows;
        
        Mat image_roi = test_image(roi);
        
        // 3. Transform it using the zero-mean, unit-variance normalisation
        Mat normalised_image_roi = zeroMeanUnitVarianceNormalisation(image_roi);
        
        // 4. Perform a pixel-wise product between the two normalised patches
        // 5. Compute its mean pixel value
        Scalar average = mean(normalised_image_roi.mul(normalised_template));
        
        // 6. This is the auto-correlation value for that pixel location
        int x = i + template_image.cols / 2;
        int y = j + template_image.rows / 2;
        autocorrelation_float.at<float>(Point(x, y)) = average[0];

        // Display an animation
        int rescaled = floor(255.0 * ((average[0] + 1) / 2.0));
        if (rescaled < 0) rescaled = 0;
        else if (rescaled > 255) rescaled = 255;
        
        Vec3b& colour = autocorrelation_uchar.at<Vec3b>(Point(x, y));
        colour.val[0] = rescaled;
        colour.val[1] = rescaled;
        colour.val[2] = rescaled;
        
        if (!(i % 5) && !(j % 5))
        {
            Mat display_roi;
            test_image.copyTo(display_roi);
            rectangle(display_roi, Point(i,j), Point(i+template_image.cols,j+template_image.rows), Scalar(255, 255, 255), 2, 2, 0);

            Mat vis;
            hconcat(display_roi,autocorrelation_uchar,vis);
            imshow("Visualisation", vis);
            waitKey(1);
        }
    }
}

// 7. The auto-correlation values are between -1 and 1.
double min_value, max_value;
minMaxLoc(autocorrelation_float, &min_value, &max_value);
cout << min_value << " " << max_value << endl;

Mat vis;
hconcat(test_image, autocorrelation_uchar, vis);
imshow("Visualisation", vis);
waitKey(1);

imwrite("../example1/autocorrelation.png", ((autocorrelation_float + 1.0) / 2.0) * 255.0);

-0.595917 0.972948


## Cross-correlation

![](../example1/autocorrelation.png)

## Apply a threshold

In [10]:
Mat binary_mask;
threshold(autocorrelation_float,
          binary_mask,
          0.5,
          255,
          THRESH_BINARY);
binary_mask.convertTo(binary_mask, CV_8UC3);
imwrite("../example1/binary_mask.png", binary_mask);

![binary_mask](../example1/binary_mask.png)

## Find every island



In [11]:
vector<vector<Point> > contours;
vector<Vec4i> hierarchy;
findContours( binary_mask, contours, hierarchy, RETR_TREE, CHAIN_APPROX_SIMPLE );
cout << contours.size() << " contours found" << endl;

86 contours found


## Draw the cells in random colours

In [12]:
RNG rng(12345);

In [13]:
Mat drawing = test_image.clone();
for( size_t i = 0; i< contours.size(); i++ )
{
    Scalar color = Scalar( rng.uniform(0, 256), rng.uniform(0,256), rng.uniform(0,256) );
    drawContours( drawing, contours, (int)i, color, 2, LINE_8, hierarchy, 0 );
}
imwrite("../example1/drawing.png", drawing);
imshow( "Contours", drawing );
waitKey(1);

![](../example1/drawing.png)

In [14]:
waitKey(0);
destroyAllWindows();